In [27]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

pd.options.display.max_columns = None  # 길어서 생략된 열 다 보이게 하기

In [3]:
events1= pd.read_parquet('/Users/hj/Documents/Four-A/dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized_divided1.parquet')
events2= pd.read_parquet('/Users/hj/Documents/Four-A/dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized_divided2.parquet')
events = pd.concat([events1, events2]).sort_index()
events

,event_id,user_id,sequence_number,session_id,event_created_at,ip_address,city,state,postal_code,browser,event_traffic_source,uri,event_type
0,2198523,<NA>,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel
1,1773216,<NA>,3,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00,85.114.141.79,Santa Isabel,São Paulo,07500-000,Safari,Adwords,/cancel,cancel
2,2380515,<NA>,3,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00,169.250.255.132,Mairiporã,São Paulo,07600-000,IE,Adwords,/cancel,cancel
3,2250597,<NA>,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel
4,1834446,<NA>,3,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00,161.114.4.174,São Paulo,São Paulo,09581-680,Chrome,Email,/cancel,cancel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431958,822590,62906,5,12ae51e7-76b3-4560-9335-3ea762eae60e,2022-04-07 01:55:02+00:00,52.92.60.25,St Helens,England,WA9,Firefox,Adwords,/purchase,purchase
2431959,43178,3333,5,e472549e-273e-402e-ae5f-38ece25699c2,2023-09-26 10:25:28+00:00,97.78.106.249,Orrell,England,WN5,IE,Email,/purchase,purchase
2431960,553230,42386,10,6a7ba027-2af2-46a3-ae05-294a264840cf,2023-07-31 15:11:10+00:00,147.252.137.75,Orrell,England,WN5,Chrome,Facebook,/purchase,purchase
2431961,1015190,77546,5,29e5960c-214b-456e-8c38-fe6eb8ff1ec8,2024-01-02 14:26:38+00:00,112.209.120.82,Wolverhampton,England,WV6,Chrome,Adwords,/purchase,purchase


In [5]:
events = events[events['event_created_at']<"2023-12-01"]
events

,event_id,user_id,sequence_number,session_id,event_created_at,ip_address,city,state,postal_code,browser,event_traffic_source,uri,event_type
0,2198523,<NA>,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel
1,1773216,<NA>,3,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00,85.114.141.79,Santa Isabel,São Paulo,07500-000,Safari,Adwords,/cancel,cancel
2,2380515,<NA>,3,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00,169.250.255.132,Mairiporã,São Paulo,07600-000,IE,Adwords,/cancel,cancel
3,2250597,<NA>,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel
4,1834446,<NA>,3,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00,161.114.4.174,São Paulo,São Paulo,09581-680,Chrome,Email,/cancel,cancel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431957,964398,73628,5,959d7ba8-e279-4581-97ce-bc8ef6d754db,2022-04-07 05:50:49+00:00,176.114.208.145,Warrington,England,WA5,Chrome,Adwords,/purchase,purchase
2431958,822590,62906,5,12ae51e7-76b3-4560-9335-3ea762eae60e,2022-04-07 01:55:02+00:00,52.92.60.25,St Helens,England,WA9,Firefox,Adwords,/purchase,purchase
2431959,43178,3333,5,e472549e-273e-402e-ae5f-38ece25699c2,2023-09-26 10:25:28+00:00,97.78.106.249,Orrell,England,WN5,IE,Email,/purchase,purchase
2431960,553230,42386,10,6a7ba027-2af2-46a3-ae05-294a264840cf,2023-07-31 15:11:10+00:00,147.252.137.75,Orrell,England,WN5,Chrome,Facebook,/purchase,purchase


In [7]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2220767 entries, 0 to 2431962
Data columns (total 13 columns):
 #   Column                Dtype              
---  ------                -----              
 0   event_id              Int32              
 1   user_id               Int32              
 2   sequence_number       Int8               
 3   session_id            object             
 4   event_created_at      datetime64[ns, UTC]
 5   ip_address            object             
 6   city                  category           
 7   state                 category           
 8   postal_code           object             
 9   browser               category           
 10  event_traffic_source  category           
 11  uri                   object             
 12  event_type            category           
dtypes: Int32(2), Int8(1), category(5), datetime64[ns, UTC](1), object(4)
memory usage: 142.2+ MB


In [18]:
events.isnull().sum()

event_id                      0
user_id                 1096789
sequence_number               0
session_id                    0
event_created_at              0
ip_address                    0
city                      20733
state                         0
postal_code                   0
browser                       0
event_traffic_source          0
uri                           0
event_type                    0
year                          0
month                         0
day                           0
dtype: int64

In [24]:
events['year'] = events['event_created_at'].dt.year
events['month'] = events['event_created_at'].dt.month
events['day'] = events['event_created_at'].dt.day
events['date'] = pd.to_datetime(events['event_created_at']).dt.date
events

/var/folders/qv/tqmmyp0s5dj996k_tcvqmtqm0000gn/T/ipykernel_16187/760177543.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['year'] = events['event_created_at'].dt.year
/var/folders/qv/tqmmyp0s5dj996k_tcvqmtqm0000gn/T/ipykernel_16187/760177543.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['month'] = events['event_created_at'].dt.month
/var/folders/qv/tqmmyp0s5dj996k_tcvqmtqm0000gn/T/ipykernel_16187/760177543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,event_id,user_id,sequence_number,session_id,event_created_at,ip_address,city,state,postal_code,browser,event_traffic_source,uri,event_type,year,month,day,date_ymd,date
0,2198523,<NA>,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel,2021,6,17,2021-06-17 17:30:00+00:00,2021-06-17
1,1773216,<NA>,3,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00,85.114.141.79,Santa Isabel,São Paulo,07500-000,Safari,Adwords,/cancel,cancel,2020,8,7,2020-08-07 08:41:00+00:00,2020-08-07
2,2380515,<NA>,3,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00,169.250.255.132,Mairiporã,São Paulo,07600-000,IE,Adwords,/cancel,cancel,2021,2,15,2021-02-15 18:48:00+00:00,2021-02-15
3,2250597,<NA>,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel,2022,3,30,2022-03-30 10:56:00+00:00,2022-03-30
4,1834446,<NA>,3,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00,161.114.4.174,São Paulo,São Paulo,09581-680,Chrome,Email,/cancel,cancel,2019,9,5,2019-09-05 01:18:00+00:00,2019-09-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431957,964398,73628,5,959d7ba8-e279-4581-97ce-bc8ef6d754db,2022-04-07 05:50:49+00:00,176.114.208.145,Warrington,England,WA5,Chrome,Adwords,/purchase,purchase,2022,4,7,2022-04-07 05:50:49+00:00,2022-04-07
2431958,822590,62906,5,12ae51e7-76b3-4560-9335-3ea762eae60e,2022-04-07 01:55:02+00:00,52.92.60.25,St Helens,England,WA9,Firefox,Adwords,/purchase,purchase,2022,4,7,2022-04-07 01:55:02+00:00,2022-04-07
2431959,43178,3333,5,e472549e-273e-402e-ae5f-38ece25699c2,2023-09-26 10:25:28+00:00,97.78.106.249,Orrell,England,WN5,IE,Email,/purchase,purchase,2023,9,26,2023-09-26 10:25:28+00:00,2023-09-26
2431960,553230,42386,10,6a7ba027-2af2-46a3-ae05-294a264840cf,2023-07-31 15:11:10+00:00,147.252.137.75,Orrell,England,WN5,Chrome,Facebook,/purchase,purchase,2023,7,31,2023-07-31 15:11:10+00:00,2023-07-31


In [15]:
# 월별 유저 행동 카운트 
"""
-> 보기만 하고 안산 유저
-> 고객 구매까지의 여정이 짧음 
"""

pd.DataFrame(events['month'].value_counts())

,count
month,
11,223860
10,220725
9,202876
8,201212
7,196633
5,185615
6,182695
3,173776
4,171811


In [17]:
# 월별 사용자 이용수  (유니크 id)

unique_user_counts = events.groupby('month')['user_id'].nunique()
unique_user_counts.sort_values()

month
12     6340
2      6353
1      6837
3      7513
4      7666
5      8393
6      8674
7      9459
8     10007
9     10409
10    11559
11    12356
Name: user_id, dtype: int64

## DAU (일간 활성 사용자수) 추이

In [25]:
dau = events.groupby('date')[['user_id']].nunique().reset_index().rename({'user_id':'dau'}, axis=1)
dau

,date,dau
0,2019-01-02,0
1,2019-01-03,0
2,2019-01-04,0
3,2019-01-05,0
4,2019-01-06,1
...,...,...
1789,2023-11-26,388
1790,2023-11-27,361
1791,2023-11-28,357
1792,2023-11-29,364


In [65]:
fig = px.line(data_frame = dau, x='date', y='dau', title='DAU 추이')
fig.show()

In [36]:
dau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1794 entries, 0 to 1793
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    1794 non-null   object
 1   dau     1794 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 28.2+ KB


In [41]:
dau['date'] = pd.to_datetime(dau['date'])

dau['day_of_week'] = dau['date'].dt.day_name()
dau['day_of_week1'] = dau['date'].dt.day_of_week
dau

,date,dau,day_of_week,day_of_week1
0,2019-01-02,0,Wednesday,2
1,2019-01-03,0,Thursday,3
2,2019-01-04,0,Friday,4
3,2019-01-05,0,Saturday,5
4,2019-01-06,1,Sunday,6
...,...,...,...,...
1789,2023-11-26,388,Sunday,6
1790,2023-11-27,361,Monday,0
1791,2023-11-28,357,Tuesday,1
1792,2023-11-29,364,Wednesday,2


In [42]:
## 요일별 DAU 

avg_dau_by_dow = dau.groupby(['day_of_week', 'day_of_week1'])[['dau']].mean().reset_index()
avg_dau_by_dow.sort_values('day_of_week1', inplace=True)
avg_dau_by_dow

,day_of_week,day_of_week1,dau
1,Monday,0,91.718750
5,Tuesday,1,92.070312
6,Wednesday,2,91.964981
4,Thursday,3,92.509728
0,Friday,4,91.066406
2,Saturday,5,91.187500
3,Sunday,6,92.191406


In [44]:
fig = px.bar(data_frame=avg_dau_by_dow, x='day_of_week', y='dau', title='요일별 DAU 평균', width=700, height=500)
fig.show()

## 사이트 체류시간 평균

In [73]:
events.head()

,event_id,user_id,sequence_number,session_id,event_created_at,ip_address,city,state,postal_code,browser,event_traffic_source,uri,event_type,year,month,day,date_ymd,date
0,2198523,<NA>,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel,2021,6,17,2021-06-17 17:30:00+00:00,2021-06-17
1,1773216,<NA>,3,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00,85.114.141.79,Santa Isabel,São Paulo,07500-000,Safari,Adwords,/cancel,cancel,2020,8,7,2020-08-07 08:41:00+00:00,2020-08-07
2,2380515,<NA>,3,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00,169.250.255.132,Mairiporã,São Paulo,07600-000,IE,Adwords,/cancel,cancel,2021,2,15,2021-02-15 18:48:00+00:00,2021-02-15
3,2250597,<NA>,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel,2022,3,30,2022-03-30 10:56:00+00:00,2022-03-30
4,1834446,<NA>,3,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00,161.114.4.174,São Paulo,São Paulo,09581-680,Chrome,Email,/cancel,cancel,2019,9,5,2019-09-05 01:18:00+00:00,2019-09-05


In [74]:
events.query('session_id == "83889ed2-2adc-4b9a-af5d-154f6998e778"').sort_values('event_created_at')

,event_id,user_id,sequence_number,session_id,event_created_at,ip_address,city,state,postal_code,browser,event_traffic_source,uri,event_type,year,month,day,date_ymd,date
1380245,2198521,<NA>,1,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:10:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/product/16375,product,2021,6,17,2021-06-17 17:10:00+00:00,2021-06-17
2259680,2198522,<NA>,2,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:24:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cart,cart,2021,6,17,2021-06-17 17:24:00+00:00,2021-06-17
0,2198523,<NA>,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel,2021,6,17,2021-06-17 17:30:00+00:00,2021-06-17


In [78]:
events[(events['event_id'] >= 2198524) & (events['event_id'] < 2198535)].sort_values('event_id')

,event_id,user_id,sequence_number,session_id,event_created_at,ip_address,city,state,postal_code,browser,event_traffic_source,uri,event_type,year,month,day,date_ymd,date
1135697,2198524,<NA>,1,8ecb870d-9f42-4967-a2be-27bb91d1b90f,2023-11-09 11:24:00+00:00,177.193.218.240,Osterode am Harz,Niedersachsen,37520,Chrome,YouTube,/product/3411,product,2023,11,9,2023-11-09 11:24:00+00:00,2023-11-09
1121431,2198525,<NA>,2,8ecb870d-9f42-4967-a2be-27bb91d1b90f,2023-11-09 11:38:00+00:00,177.193.218.240,Osterode am Harz,Niedersachsen,37520,Chrome,YouTube,/cart,cart,2023,11,9,2023-11-09 11:38:00+00:00,2023-11-09
1663869,2198526,<NA>,3,8ecb870d-9f42-4967-a2be-27bb91d1b90f,2023-11-09 12:06:00+00:00,177.193.218.240,Osterode am Harz,Niedersachsen,37520,Chrome,YouTube,/cancel,cancel,2023,11,9,2023-11-09 12:06:00+00:00,2023-11-09
1948667,2198527,<NA>,1,f4e27ea2-9d78-41ed-a12d-b671324efbec,2021-10-28 14:43:00+00:00,111.193.61.121,San Tan Valley,Arizona,85140,Firefox,Organic,/department/women/category/leggings/brand/luxu...,department,2021,10,28,2021-10-28 14:43:00+00:00,2021-10-28
1015576,2198528,<NA>,2,f4e27ea2-9d78-41ed-a12d-b671324efbec,2021-10-28 14:43:00+00:00,111.193.61.121,San Tan Valley,Arizona,85140,Firefox,Organic,/product/5793,product,2021,10,28,2021-10-28 14:43:00+00:00,2021-10-28
2249001,2198529,<NA>,1,9fd8f1f7-caae-4e48-945e-848cffc30735,2019-10-07 16:07:00+00:00,72.76.190.217,National city,Tokyo,186-0004,Other,Adwords,/product/18065,product,2019,10,7,2019-10-07 16:07:00+00:00,2019-10-07
662528,2198530,<NA>,1,f5911105-38c1-404a-a8ba-f1697eb21eac,2021-01-12 15:57:00+00:00,81.213.8.124,Decatur,Georgia,30030,Firefox,Email,/product/26836,product,2021,1,12,2021-01-12 15:57:00+00:00,2021-01-12
633922,2198531,<NA>,1,f823b8c9-7e79-4249-a5fe-2666e9433b81,2021-11-28 07:58:00+00:00,34.139.123.246,Chula Vista,California,91913,Safari,Facebook,/department/men/category/suits&sportcoats/bran...,department,2021,11,28,2021-11-28 07:58:00+00:00,2021-11-28
244674,2198532,<NA>,2,f823b8c9-7e79-4249-a5fe-2666e9433b81,2021-11-28 08:22:00+00:00,34.139.123.246,Chula Vista,California,91913,Safari,Facebook,/product/20453,product,2021,11,28,2021-11-28 08:22:00+00:00,2021-11-28
876574,2198533,<NA>,3,f823b8c9-7e79-4249-a5fe-2666e9433b81,2021-11-28 08:50:00+00:00,34.139.123.246,Chula Vista,California,91913,Safari,Facebook,/cart,cart,2021,11,28,2021-11-28 08:50:00+00:00,2021-11-28


In [51]:
print(events.query('session_id == "959d7ba8-e279-4581-97ce-bc8ef6d754db"')['event_created_at'].max())
print(events.query('session_id == "959d7ba8-e279-4581-97ce-bc8ef6d754db"')['event_created_at'].min())
print(events.query('session_id == "959d7ba8-e279-4581-97ce-bc8ef6d754db"')['event_created_at'].max() - events.query('session_id == "959d7ba8-e279-4581-97ce-bc8ef6d754db"')['event_created_at'].min())


2022-04-07 05:50:49+00:00
2022-04-07 05:45:12+00:00
0 days 00:05:37


In [52]:
duration = events.groupby('session_id')[['event_created_at']].agg(['max', 'min']).reset_index()
duration['duration'] = duration['event_created_at']['max'] - duration['event_created_at']['min']

In [54]:
duration.columns = ['session_id', 'max', 'min', 'duration']
duration

,session_id,max,min,duration
0,00000763-a855-4ad0-a95c-b160e749b272,2021-10-01 03:52:00+00:00,2021-10-01 03:26:00+00:00,0 days 00:26:00
1,0000364a-ce41-46f1-89d6-3f8704af77db,2022-08-22 10:52:42+00:00,2022-08-22 10:46:40+00:00,0 days 00:06:02
2,00004b15-f2d4-4687-b4c1-fc9ce336d39a,2022-06-11 01:54:00+00:00,2022-06-11 01:28:00+00:00,0 days 00:26:00
3,00004cf0-0d54-4347-8b0c-dccc700a2c96,2021-12-28 03:32:00+00:00,2021-12-28 03:11:00+00:00,0 days 00:21:00
4,00005f74-03cb-40ed-b254-364c38c79104,2019-08-02 14:49:00+00:00,2019-08-02 14:40:00+00:00,0 days 00:09:00
...,...,...,...,...
643588,ffff7645-26f7-410f-8498-0c601d74c67a,2019-11-05 15:39:00+00:00,2019-11-05 15:00:00+00:00,0 days 00:39:00
643589,ffff8086-da2c-4045-94ba-453437cd3289,2020-03-20 06:06:00+00:00,2020-03-20 06:06:00+00:00,0 days 00:00:00
643590,ffffa9aa-4345-4c2a-b389-8fd58844472e,2021-07-26 15:49:00+00:00,2021-07-26 15:45:00+00:00,0 days 00:04:00
643591,ffffc3b7-9bc3-44d0-935e-cfaf11c91a0e,2019-07-10 07:21:00+00:00,2019-07-10 07:02:00+00:00,0 days 00:19:00


- 체류시간 평균

In [56]:
duration['duration'].mean()

Timedelta('0 days 06:16:15.696093649')

### 조회만 한 유저, 카트에 담은 유저, 구매까지 한 유저 별 체류 시간 비교

In [59]:
session_pivot = pd.pivot_table(data=events, index='session_id', columns='event_type', values='event_created_at', aggfunc='count').reset_index().fillna(0)
session_pivot

/var/folders/qv/tqmmyp0s5dj996k_tcvqmtqm0000gn/T/ipykernel_16187/745961524.py:1: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



event_type,session_id,cancel,cart,department,home,product,purchase
0,00000763-a855-4ad0-a95c-b160e749b272,0,1,1,0,1,0
1,0000364a-ce41-46f1-89d6-3f8704af77db,0,1,1,1,1,1
2,00004b15-f2d4-4687-b4c1-fc9ce336d39a,0,1,1,0,1,0
3,00004cf0-0d54-4347-8b0c-dccc700a2c96,0,1,1,0,1,0
4,00005f74-03cb-40ed-b254-364c38c79104,0,0,1,0,1,0
...,...,...,...,...,...,...,...
643588,ffff7645-26f7-410f-8498-0c601d74c67a,1,1,0,0,1,0
643589,ffff8086-da2c-4045-94ba-453437cd3289,0,0,0,0,1,0
643590,ffffa9aa-4345-4c2a-b389-8fd58844472e,0,0,1,0,1,0
643591,ffffc3b7-9bc3-44d0-935e-cfaf11c91a0e,0,1,1,0,1,0


In [85]:
max_purchase = session_pivot['purchase'].max()
print("purchase 열의 최대값:", max_purchase)


purchase 열의 최대값: 1


In [60]:
cart_session = list(session_pivot.query('cart > 0')['session_id'])
purchased_session = list(session_pivot.query('purchase > 0')['session_id'])

In [64]:
view_session_avg_duration = duration.query('session_id not in @cart_session and session_id not in @purchased_session')['duration'].mean()
cart_session_avg_duration = duration.query('session_id in @cart_session')['duration'].mean()
purchased_session_avg_duration = duration.query('session_id in @purchased_session')['duration'].mean()

print(f"조회만 한 유저의 평균 체류시간 : {view_session_avg_duration}")
print(f"카트에 담은 유저의 평균 체류시간 : {cart_session_avg_duration}")
print(f"구매까지 한 유저의 평균 체류시간 : {purchased_session_avg_duration}")

조회만 한 유저의 평균 체류시간 : 0 days 00:07:15.212859599
카트에 담은 유저의 평균 체류시간 : 0 days 10:00:16.610015606
구매까지 한 유저의 평균 체류시간 : 1 days 00:54:43.280229108


## 어느 단계에서 유저들이 가장 많이 이탈하나?

In [69]:
funnel = session_pivot[['home', 'department', 'product','cart', 'cancel', 'purchase']].sum().to_frame().reset_index()
funnel.columns = ['event_type', 'count']
funnel = funnel.query('event_type != "cancel"')
funnel

,event_type,count
0,home,75442
1,department,540730
2,product,784621
3,cart,541504
5,purchase,156083


In [90]:
fig = px.funnel(data_frame=funnel, x='event_type', y='count', title='Funnel aka 운동기구')
fig.update_traces(texttemplate="%{value:,.0f}")
fig.show()


In [83]:
sequence_1 = events[events['sequence_number'] == 1].groupby('event_type').size().sort_values(ascending=False)
sequence_1

/var/folders/qv/tqmmyp0s5dj996k_tcvqmtqm0000gn/T/ipykernel_16187/3593796189.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



event_type
department    324259
product       243892
home           75442
cancel             0
cart               0
purchase           0
dtype: int64

- department와 product는 광고 캠페인에 사용된 랜딩페이지 일 가능성 높음
- 랜딩페이지(온라인 광고를 클릭하면 바로 연결되는 페이지)

In [88]:
canceled_data= events[events['event_type']=='cancel']
canceled_data

,event_id,user_id,sequence_number,session_id,event_created_at,ip_address,city,state,postal_code,browser,event_traffic_source,uri,event_type,year,month,day,date_ymd,date
0,2198523,<NA>,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel,2021,6,17,2021-06-17 17:30:00+00:00,2021-06-17
1,1773216,<NA>,3,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00,85.114.141.79,Santa Isabel,São Paulo,07500-000,Safari,Adwords,/cancel,cancel,2020,8,7,2020-08-07 08:41:00+00:00,2020-08-07
2,2380515,<NA>,3,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00,169.250.255.132,Mairiporã,São Paulo,07600-000,IE,Adwords,/cancel,cancel,2021,2,15,2021-02-15 18:48:00+00:00,2021-02-15
3,2250597,<NA>,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel,2022,3,30,2022-03-30 10:56:00+00:00,2022-03-30
4,1834446,<NA>,3,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00,161.114.4.174,São Paulo,São Paulo,09581-680,Chrome,Email,/cancel,cancel,2019,9,5,2019-09-05 01:18:00+00:00,2019-09-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408413,1773586,<NA>,3,6573bb67-f346-4448-b98f-167fbed8bf52,2023-07-07 12:00:00+00:00,10.29.170.67,London,England,W3,Chrome,Adwords,/cancel,cancel,2023,7,7,2023-07-07 12:00:00+00:00,2023-07-07
2408415,2219661,<NA>,3,244b19b9-3570-4b0c-bd4d-ca923c157f7f,2020-10-04 17:23:00+00:00,96.30.193.225,Wakefield,England,WF2,Firefox,Email,/cancel,cancel,2020,10,4,2020-10-04 17:23:00+00:00,2020-10-04
2408416,1521021,<NA>,3,b1d4e0ed-685b-4365-8933-84817ad9730f,2023-03-17 11:18:00+00:00,113.98.116.93,Droitwich Spa,England,WR9,Safari,YouTube,/cancel,cancel,2023,3,17,2023-03-17 11:18:00+00:00,2023-03-17
2408417,1628856,<NA>,3,0ce673da-fad8-4f2e-b5e1-e9dc4b8c40a4,2021-12-25 17:57:00+00:00,41.128.250.236,Wednesbury,England,WS10,Chrome,Email,/cancel,cancel,2021,12,25,2021-12-25 17:57:00+00:00,2021-12-25


In [87]:
canceled_data.isnull().sum()

event_id                     0
user_id                 122387
sequence_number              0
session_id                   0
event_created_at             0
ip_address                   0
city                      1141
state                        0
postal_code                  0
browser                      0
event_traffic_source         0
uri                          0
event_type                   0
year                         0
month                        0
day                          0
date_ymd                     0
date                         0
dtype: int64

In [89]:
canceled_data[canceled_data['user_id'].notnull()]

,event_id,user_id,sequence_number,session_id,event_created_at,ip_address,city,state,postal_code,browser,event_traffic_source,uri,event_type,year,month,day,date_ymd,date


- cancel되면 id 널값으로 바뀜